In [ ]:
# outcome 제하고 BMI 예측하는 회귀분석 코드

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1) 데이터 로드
df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/data/diabetes.csv")

# 2) Outcome 제거
if 'Outcome' in df.columns:
    df = df.drop(columns=['Outcome'])

# 3) BMI 타깃의 0/NaN 행 제거
df = df[~((df['BMI'] == 0) | (df['BMI'].isna()))].reset_index(drop=True)

# 4) 입력 변수 결측/0 처리 (예: Glucose 등)
cols_to_fix = [c for c in ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin'] if c in df.columns]
for c in cols_to_fix:
    df[c] = df[c].replace(0, np.nan)
    df[c] = df[c].fillna(df[c].median())

# 5) X, y 분리
X = df.drop(columns=['BMI'])
y = df['BMI']

# 6) 학습/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7) 스케일링 — **반드시 여기서 생성** (이 줄이 없으면 NameError 발생)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)   # <-- fit_transform: 학습 데이터로 fit + transform
X_test_scaled = scaler.transform(X_test)         # <-- transform만

# (옵션) 변수 존재 확인
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)

# 8) 모델 정의 및 학습
models = {
    'LinearRegression': LinearRegression(),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=200, random_state=42)
}

def regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'mse': mse, 'rmse': rmse, 'mae': mae, 'r2': r2}

results = {}
for name, model in models.items():
    model.fit(X_train_scaled, y_train)           # X_train_scaled가 반드시 존재해야 함
    y_pred = model.predict(X_test_scaled)
    results[name] = regression_metrics(y_test, y_pred)
    print(f"\n{name}: RMSE={results[name]['rmse']:.4f}, MAE={results[name]['mae']:.4f}, R2={results[name]['r2']:.4f}")
